In [1]:
import doctest
import re
from dataclasses import dataclass
from math import prod as power
from typing import List, Tuple

In [2]:
DATA = "input.txt"

# Part 1

In [3]:
@dataclass
class Draw:
    R: int = 0
    G: int = 0
    B: int = 0

@dataclass
class Game:
    id: int
    draws: List[Draw]

In [4]:
def parse_game(line: str) -> Game:
    """

    Example:

        >>> parse_game("Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green")
        Game(id=1, draws=[Draw(R=4, G=0, B=3), Draw(R=1, G=2, B=6), Draw(R=0, G=2, B=0)])

        >>> parse_game("Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue")
        Game(id=2, draws=[Draw(R=0, G=2, B=1), Draw(R=1, G=3, B=4), Draw(R=0, G=1, B=1)])

        >>> parse_game("Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red")
        Game(id=3, draws=[Draw(R=20, G=8, B=6), Draw(R=4, G=13, B=5), Draw(R=1, G=5, B=0)])

        >>> parse_game("Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red")
        Game(id=4, draws=[Draw(R=3, G=1, B=6), Draw(R=6, G=3, B=0), Draw(R=14, G=3, B=15)])

        >>> parse_game("Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green")
        Game(id=5, draws=[Draw(R=6, G=3, B=1), Draw(R=1, G=2, B=2)])
    """
    game_draws = re.match("Game (\d+):(.*)", line)
    game_id = int(game_draws.group(1))
    ps = [re.compile(f".* (\d+) {color}.*") for color in ["red", "green", "blue"]]
    draws = []
    for draw in game_draws.group(2).split(";"):
        r, g, b = [p.match(draw) for p in ps]
        draws.append(
            Draw(
                R=int(r.group(1)) if r else 0,
                G=int(g.group(1)) if g else 0,
                B=int(b.group(1)) if b else 0
            )
        )
    return Game(id=game_id, draws=draws)

In [5]:
def is_possible(game: Game, red: int, green: int, blue: int) -> bool:
    """Return True if ``game`` was possible for given bag cubes.

    Example:

        >>> game = parse_game("Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red")
        >>> is_possible(game, red=12, green=13, blue=14)
        False
    """
    return all(
        (draw.R <= red) and (draw.G <= green) and (draw.B <= blue)
        for draw in game.draws
    )

In [6]:
doctest.testmod()

TestResults(failed=0, attempted=7)

In [7]:
games = []

with open(DATA, "r") as f:
    for line in f:
        games.append(parse_game(line))

sum(g.id for g in games if is_possible(g, red=12, green=13, blue=14))

2439

## Part 2

In [8]:
def minimum_cubes(game: Game) -> Tuple[int, int, int]:
    """Return the minimum number of cubes required to play a game.

    Example:

        >>> game = parse_game("Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green")
        >>> minimum_cubes(game)
        (4, 2, 6)

        >>> game = parse_game("Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue")
        >>> minimum_cubes(game)
        (1, 3, 4)

        >>> game = parse_game("Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red")
        >>> minimum_cubes(game)
        (20, 13, 6)

        >>> game = parse_game("Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red")
        >>> minimum_cubes(game)
        (14, 3, 15)

        >>> game = parse_game("Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green")
        >>> minimum_cubes(game)
        (6, 3, 2)
    """
    r, g, b = None, None, None
    for draw in game.draws:
        r = max(r, draw.R) if r is not None else draw.R
        g = max(g, draw.G) if g is not None else draw.G
        b = max(b, draw.B) if b is not None else draw.B
    return r, g, b

In [9]:
doctest.testmod()

TestResults(failed=0, attempted=17)

In [10]:
games = []

with open(DATA, "r") as f:
    for line in f:
        games.append(parse_game(line))

sum(power(minimum_cubes(g)) for g in games)

63711